In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
/;


....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


In [3]:
my $input = $practice_input;

sub sim_step {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));
    my %states;
    my @states_list;
    my $is_loop = 0;
    while ($input =~ s/\.([^\x00]{$shape[1]})\^/^${1}\./s
            || $input =~ s/#([^\x00]{$shape[1]})\^/#${1}>/s
            || $input =~ s/>\./\.>/s
            || $input =~ s/>#/v#/s
            || $input =~ s/v([^\x00]{$shape[1]})\./\.${1}v/s
            || $input =~ s/v([^\x00]{$shape[1]})#/<${1}#/s
            || $input =~ s/\.</<\./s
            || $input =~ s/#</#^/s) {
        if (exists $states{$input}) {
            $is_loop = 1;
            last;
        }
        ($states{$input}) = ();
        push @states_list, $input;
        # say "step:\n", $input;
    }
    return $is_loop, @states_list;
    # return $is_loop;
}

sub quick_sim_step {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));
    my %states;
    my @states_list;
    my $is_loop = 0;
    my $row = "[\x00-\xff]{$shape[1]}";
    while (
            $input =~ s/>(\.*)([#O])/${1}v${2}/s
            || $input =~ s/v($row(?:\.$row)*)\.($row)([#O])/\.${1}<${2}${3}/s
            || $input =~ s/([#O])(\.*)</${1}^${2}/s
            || $input =~ s/([#O])($row)\.($row(\.$row)*)\^/${1}${2}>${3}\./s

            || $input =~ s/>(\.+)/${1}>/s
            || $input =~ s/v($row(?:\.$row)*)\./\.${1}v/s
            || $input =~ s/v($row)([#O])/<${1}${2}/s
            || $input =~ s/([#O])($row)\^/${1}${2}>/s
            || $input =~ s/(\.+)</<${1}/s
            || $input =~ s/\.($row(\.$row)*)\^/^${1}\./s
            ) {
        if (exists $states{$input}) {
            $is_loop = 1;
            last;
        }
        ($states{$input}) = ();
        push @states_list, $input;
        # say "step:\n", $input;
    }
    return $is_loop, @states_list;
    # return $is_loop;
}

sub add_obstacle {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));
    if ($input =~ s/\.([^\x00]{$shape[1]})\^/O${1}^/s
            || $input =~ s/>\./>O/s
            || $input =~ s/v([^\x00]{$shape[1]})\./v${1}O/s
            || $input =~ s/\.</O</s) {
        return $input;
    } else {
        return;
    }
}

sub reset_caret_position {
    my ($input, $modified_input) = @_;
    return unless defined $modified_input;
    
    my $index = index $input, '^';
    die "wat: $index" if $index < 0;
    $modified_input =~ s/[<>^v]/./g;
    substr($modified_input, $index, 1) = '^';
    return $modified_input;
}


sub or_product { my $s = ''; foreach (@_) { $s |= $_ } $s }
sub swap_dots { $_[0] =~ s/\./-/gr }
sub swap_vertical_dots {
    my ($s, $row_length) = @_;
    return join '', map { length > 1 ? $_ : y/-\./+|/r } grep length > 0, unpack("(A${row_length}A1)*", $s);
}

sub render_steps {
    my ($map) = @_;
    my ($is_loop, @map_steps) = quick_sim_step($map);
    
    my @shape = shape(parse_2d_map_array($map));
    my $row = "[\x00-\xff]{$shape[1]}";
    
    my $combined_map = or_product(map { s/\./\x00/grs } $map, @map_steps) =~ s/\x00/./grs;
    $combined_map =~ s/>([\.>]+)v/'>' . swap_dots($1) . 'v'/ge;
    $combined_map =~ s/\^([\.\^<]+)</'^' . swap_dots($1) . '<'/ge;
    while ($combined_map =~ s/v($row(?:[\.\-]$row)+)(<|v)/'v' . swap_vertical_dots($1, $shape[1]) . $2/ge) {}
    while ($combined_map =~ s/(>|\^)($row(?:[\.\-]$row)+)\^/$1 . swap_vertical_dots($2, $shape[1]) . '^'/ge) {}

    return $combined_map;
}

my ($is_loop, @steps) = sim_step($input);
my @found_loops;
foreach (@steps) {
    if (my $modified_map = reset_caret_position($input, add_obstacle($_))) {
        # say "can add obstacle:\n$modified_map";
        my ($is_another_loop) = quick_sim_step($modified_map);
        if ($is_another_loop) {
            # say "possible loop:\n$modified_map";
            push @found_loops, $modified_map;
            # $loop_count++;
        }
    }
}
# say "loop:\n $_\n", render_steps($_) foreach uniq @found_loops;
my $loop_count = uniq @found_loops;
say render_steps($input);
say "loop_count: $loop_count";
# say Dumper [ quick_sim_step($input) ];



....#.....
....>---v#
....|...|.
..#.|...|.
..>-+-v#|.
..|.|.|.|.
.#^-^-+-<.
.>----+v#.
#^----<|..
......#v..

loop_count: 6


1

In [4]:
my $input = get_challenge('2024/day/6/input');


In [5]:
my $start = time;
my ($is_loop, @steps) = sim_step($input);
say "time: ", time - $start;

time: 16


1

In [6]:
my $start = time;

my @found_loops = ();
my @found_not_loops = ();
foreach (@steps) {
    if (my $modified_map = reset_caret_position($input, add_obstacle($_))) {
        my ($is_another_loop) = quick_sim_step($modified_map);
        if ($is_another_loop) {
            push @found_loops, $modified_map;
        } else {
            push @found_not_loops, $modified_map;
        }
    }
}
my $loop_count = uniq @found_loops;

# say Dumper [ quick_sim_step($input) ];
say "time: ", time - $start;
say "loop_count: $loop_count";
# writefile('.exp/day6_steps', join "\n\n\n", @steps);
# my $res = process_day6_part1($input);

time: 594
loop_count: 2008


1

In [7]:

sub appendfile { local $/; my $file = IO::File->new($_[0], 'a'); $file->print($_[1]) }

my @unique_loops = @found_loops;
writefile('.exp/day6_loops_rendered', '');
for my $i (0 .. $#unique_loops) {
    appendfile('.exp/day6_loops_rendered', "\n\n>known loop [$i]:\n\n" . render_steps($unique_loops[$i]));
}

writefile('.exp/day6_notloops_rendered', '');
for my $i (0 .. $#found_not_loops) {
    appendfile('.exp/day6_notloops_rendered', "\n\n>known loop [$i]:\n\n" . render_steps($found_not_loops[$i]));
}

0


1

In [25]:
my $start = time;
my $loop_count = 0;
foreach (@steps) {
    if (my $modified_map = add_obstacle($_)) {
        # say "can add obstacle:\n$modified_map";
        my ($is_another_loop) = quick_sim_step($modified_map);
        if ($is_another_loop) {
            # say "possible loop:\n$modified_map";
            $loop_count++;
        }
    }
}
say "time: ", time - $start;
say "loop_count: $loop_count";
# writefile('.exp/day6_steps', join "\n\n\n", @steps);
# my $res = process_day6_part1($input);

time: 307
loop_count: 2258


1

In [7]:
# # more than 500, less than 2258
# not 2014
say post_answer('2024/day/6/answer', 2, 2008);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 6 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're 

1